In [1]:
#!pip install tensorflow

Defaulting to user installation because normal site-packages is not writeable



[notice] A new release of pip is available: 23.3.2 -> 24.0
[notice] To update, run: python.exe -m pip install --upgrade pip


### Paso 1: Importar librerías necesarias

In [2]:
import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Conv2D, MaxPooling2D, Flatten, Dense, Dropout
from tensorflow.keras.preprocessing.image import ImageDataGenerator

### Paso 2: Preparación y carga de datos

In [3]:
train_datagen = ImageDataGenerator(
    rescale=1./255, 
    rotation_range=20,
    width_shift_range=0.2,
    height_shift_range=0.2,
    shear_range=0.2,
    zoom_range=0.2,
    horizontal_flip=True,
    fill_mode='nearest',
    validation_split=0.2)  # Usamos el 20% de los datos para validación

train_generator = train_datagen.flow_from_directory(
    'data/',
    target_size=(150, 150),
    batch_size=32,
    class_mode='categorical',
    subset='training')  # Conjunto de entrenamiento

validation_generator = train_datagen.flow_from_directory(
    'data/',
    target_size=(150, 150),
    batch_size=32,
    class_mode='categorical',
    subset='validation')  # Conjunto de validación


Found 727 images belonging to 6 classes.
Found 179 images belonging to 6 classes.


### Paso 3: Construcción del modelo

In [4]:
model = Sequential([
    Conv2D(32, (3, 3), activation='relu', input_shape=(150, 150, 3)),
    MaxPooling2D(2, 2),
    Conv2D(64, (3, 3), activation='relu'),
    MaxPooling2D(2, 2),
    Conv2D(128, (3, 3), activation='relu'),
    MaxPooling2D(2, 2),
    Flatten(),
    Dense(512, activation='relu'),
    Dropout(0.5),
    Dense(6, activation='softmax')  # Suponiendo que hay 6 clases (0-5)
])

C:\Users\josem\AppData\Roaming\Python\Python312\site-packages\keras\src\layers\convolutional\base_conv.py:107: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)


### Paso 4: Compilar el modelo

In [5]:
model.compile(optimizer='adam',
              loss='categorical_crossentropy',
              metrics=['accuracy'])

### Paso 5: Entrenamiento del modelo

In [6]:
history = model.fit(
    train_generator,
    steps_per_epoch=train_generator.samples // train_generator.batch_size,
    validation_data=validation_generator,
    validation_steps=validation_generator.samples // validation_generator.batch_size,
    epochs=25)


Epoch 1/25


C:\Users\josem\AppData\Roaming\Python\Python312\site-packages\keras\src\trainers\data_adapters\py_dataset_adapter.py:121: UserWarning: Your `PyDataset` class should call `super().__init__(**kwargs)` in its constructor. `**kwargs` can include `workers`, `use_multiprocessing`, `max_queue_size`. Do not pass these arguments to `fit()`, as they will be ignored.
  self._warn_if_super_not_called()


22/22 ━━━━━━━━━━━━━━━━━━━━ 14s 516ms/step - accuracy: 0.1866 - loss: 1.7339 - val_accuracy: 0.2438 - val_loss: 1.6463
Epoch 2/25
 1/22 ━━━━━━━━━━━━━━━━━━━━ 4s 233ms/step - accuracy: 0.2188 - loss: 1.6830

c:\Program Files\Python312\Lib\contextlib.py:155: UserWarning: Your input ran out of data; interrupting training. Make sure that your dataset or generator can generate at least `steps_per_epoch * epochs` batches. You may need to use the `.repeat()` function when building your dataset.
  self.gen.throw(value)


22/22 ━━━━━━━━━━━━━━━━━━━━ 0s 10ms/step - accuracy: 0.2188 - loss: 1.6830 - val_accuracy: 0.2632 - val_loss: 1.6019
Epoch 3/25
22/22 ━━━━━━━━━━━━━━━━━━━━ 8s 323ms/step - accuracy: 0.2181 - loss: 1.6408 - val_accuracy: 0.2688 - val_loss: 1.5691
Epoch 4/25
22/22 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - accuracy: 0.1250 - loss: 1.6721 - val_accuracy: 0.3684 - val_loss: 1.6047
Epoch 5/25
22/22 ━━━━━━━━━━━━━━━━━━━━ 8s 305ms/step - accuracy: 0.2395 - loss: 1.5855 - val_accuracy: 0.2750 - val_loss: 1.5469
Epoch 6/25
22/22 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - accuracy: 0.3750 - loss: 1.5186 - val_accuracy: 0.2105 - val_loss: 1.5821
Epoch 7/25
22/22 ━━━━━━━━━━━━━━━━━━━━ 8s 318ms/step - accuracy: 0.2726 - loss: 1.5681 - val_accuracy: 0.3750 - val_loss: 1.4699
Epoch 8/25
22/22 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - accuracy: 0.3750 - loss: 1.4026 - val_accuracy: 0.5789 - val_loss: 1.1635
Epoch 9/25
22/22 ━━━━━━━━━━━━━━━━━━━━ 8s 340ms/step - accuracy: 0.3488 - loss: 1.5295 - val_accuracy: 0.3187 - val_loss: 1

### Paso 6: Evaluación del modelo

In [7]:
# Evaluar el modelo en el conjunto de validación
eval_result = model.evaluate(validation_generator)
print(f'Loss: {eval_result[0]}, Accuracy: {eval_result[1]}')

6/6 ━━━━━━━━━━━━━━━━━━━━ 1s 119ms/step - accuracy: 0.5487 - loss: 0.9146
Loss: 0.9302529096603394, Accuracy: 0.5642458200454712


### Paso 7: Guardar el modelo

In [8]:
# Guardar el modelo en formato HDF5
model.save('hand_number_model.h5')

# Para guardar en formato SavedModel, que es más completo, usarías:
# model.save('hand_number_model', save_format='tf')